In [1]:
import sys
sys.path.append('.')

import anndata
import time, os, sys
from datetime import datetime

import pandas as pd
import glob
pd.set_option('display.float_format', lambda x: '%.2f' % x)
import numpy as np

#import skimage.io as io
import scimap as sm
import scanpy as sc

# libraries for patch function
# Import library
import pandas as pd#
from sklearn.neighbors import BallTree
import numpy as np
from joblib import Parallel, delayed
import scipy
from functools import reduce

Running SCIMAP  2.2.11


This script runs spatial enrichment analysis (SEA) on simulated data using the scimap fork https://github.com/chiarasch/scimap

In [2]:
# Load in dataset. This is a .csv file with all samples concatenated together, either for symmetric or asymmetric data
path_to_csv = './../../../../../data/20250217_asym01_nbh2_1000dim_grid200_300iter_50swaps'
output_path_SEA_NEP = './../../../../Comparison/20250218_results_asym/SEA_delaunay_4ct_cross01.csv'

In [6]:

# read in all csv files (they all have ct, x and y coordinates) give them a new sample id column of their file names and row bind them all to one big dataframe
# Use glob to get all the CSV files in the folder
csv_files = glob.glob(os.path.join(path_to_csv, '*.csv'))

# Initialize an empty list to store individual DataFrames
data_frames = []

# Loop through the CSV files and process each one
for file in csv_files[1:10]:
    # Read the CSV file into a DataFrame
    df = pd.read_csv(file)
    sample_id = os.path.splitext(os.path.basename(file))[0]
    df['sample_id'] = sample_id
    data_frames.append(df)
    
# Concatenate all DataFrames into one big DataFrame
obs = pd.concat(data_frames, ignore_index=True)


In [4]:
# for image, show the counts of unique ct
print(obs['ct'].value_counts())

ct
1.00    4302
0.00    4038
2.00    3901
3.00    3828
Name: count, dtype: int64


In [7]:
# add marker files to it, as SpatialLDA needs them
obs['D'] = np.random.randint(1, 101, size=len(obs))
obs['E'] = np.random.randint(1, 101, size=len(obs))

# Load dataframe into anndata object
# dataframe for annotating the observations
obs = obs.astype({'ct':'string'})

# the data matrix 
X = obs[['D', 'E']]
X = X.values
adata = anndata.AnnData(X)
adata.obs = obs

adata.obs

,x,y,ct,sample_id,D,E
0,0.00,0.00,3.0,cross01_0.6_ab0_0.55_24,84,56
1,27.40,0.00,1.0,cross01_0.6_ab0_0.55_24,89,67
2,50.56,0.00,3.0,cross01_0.6_ab0_0.55_24,33,17
3,74.40,0.00,0.0,cross01_0.6_ab0_0.55_24,36,21
4,98.07,0.00,0.0,cross01_0.6_ab0_0.55_24,40,38
...,...,...,...,...,...,...
16064,161.09,167.95,2.0,cross01_0.45_ab0_0.15_78,5,84
16065,613.71,130.12,2.0,cross01_0.45_ab0_0.15_78,48,50
16066,728.30,829.93,1.0,cross01_0.45_ab0_0.15_78,51,27
16067,238.96,709.52,1.0,cross01_0.45_ab0_0.15_78,58,68


In [8]:
# run spatial interaction to look at interaction of phenotypes without motifs
sm.tl.spatial_interaction(adata, 
                          phenotype='ct', 
                          x_coordinate='x', y_coordinate='y', 
                          imageid='sample_id', 
                          #knn=8, 
                          permutation = 300,
                          method='delaunay',
                          verbose=True,
                          normalization = 'total',
                          pval_method = 'zscore',
                          label='delauany_zscore_scimap')

Processing Image: ['cross01_0.6_ab0_0.55_24']
Performing Delaunay triangulation to identify neighbours for every cell
Mapping phenotype to neighbors
Performing 300 permutations
Consolidating the permutation results
Processing Image: ['ran_ab0_0.05_5']
Performing Delaunay triangulation to identify neighbours for every cell
Mapping phenotype to neighbors
Performing 300 permutations
Consolidating the permutation results
Processing Image: ['cross01_0.45_ab0_0.15_50']
Performing Delaunay triangulation to identify neighbours for every cell
Mapping phenotype to neighbors
Performing 300 permutations
Consolidating the permutation results
Processing Image: ['cross01_0.6_ab0_0.55_30']
Performing Delaunay triangulation to identify neighbours for every cell
Mapping phenotype to neighbors
Performing 300 permutations
Consolidating the permutation results
Processing Image: ['ran_ab0_0.45_49']
Performing Delaunay triangulation to identify neighbours for every cell
Mapping phenotype to neighbors
Perform

AnnData object with n_obs × n_vars = 16069 × 2
    obs: 'x', 'y', 'ct', 'sample_id', 'D', 'E'
    uns: 'delauany_zscore_scimap'

In [9]:
save_df = adata.uns['delauany_zscore_scimap']
# Convert 'phenotype' and 'neighbour_phenotype' columns to string types
save_df['phenotype'] = save_df['phenotype'].astype(str)
save_df['neighbour_phenotype'] = save_df['neighbour_phenotype'].astype(str)

# Combine 'phenotype' and 'neighbour_phenotype' into 'new_column'
save_df['new_column'] = save_df['phenotype'] + "_" + save_df['neighbour_phenotype']
save_df = save_df.drop(columns=['phenotype', 'neighbour_phenotype'])
save_df = save_df.set_index('new_column')

# Transpose the dataframe
save_df_transposed = save_df.transpose()
save_df_transposed.columns = save_df_transposed.columns.str.replace(r"\.0",
                                                                    "",
                                                                    regex=True)
save_df_transposed = save_df_transposed[~save_df_transposed.index.str.contains('pvalue')]
save_df_transposed = save_df_transposed[~save_df_transposed.index.str.contains('count')]
save_df_transposed.index = save_df_transposed.index.str.replace(r"zscore_", "", regex=True)

# View the transposed dataframe (optional)
save_df_transposed
# save the dataframe as csv
save_df_transposed.to_csv(output_path_SEA_NEP, index=True)

new_column,0_0,0_1,0_2,0_3,1_0,1_1,1_2,1_3,2_0,2_1,2_2,2_3,3_0,3_1,3_2,3_3
cross01_0.6_ab0_0.55_24,-3.98,3.46,0.20,0.24,3.47,5.34,-3.94,-5.63,0.22,-4.30,2.66,1.83,0.16,-6.00,1.83,4.68
ran_ab0_0.05_5,0.48,-0.27,0.97,-1.17,-0.35,0.04,-0.00,0.28,1.08,-0.05,-2.76,1.66,-1.15,0.24,1.90,-0.89
cross01_0.45_ab0_0.15_50,-0.53,1.64,-0.49,-0.59,1.72,1.52,-1.00,-2.21,-0.61,-1.00,1.21,0.41,-0.57,-2.28,0.32,2.48
cross01_0.6_ab0_0.55_30,-2.89,3.89,0.00,-1.22,3.70,4.84,-6.10,-3.96,0.00,-5.56,3.02,3.76,-1.24,-4.00,3.59,2.60
ran_ab0_0.45_49,0.37,-0.22,0.54,-0.75,-0.16,-0.47,-0.95,1.60,0.44,-0.92,0.93,-0.48,-0.68,1.68,-0.53,-0.50
ran_ab0_0.45_61,-0.64,0.90,0.29,-0.48,1.03,-2.16,1.17,-0.19,0.26,1.32,-2.38,0.57,-0.54,-0.26,0.64,0.14
cross01_0.6_ab0_0.25_100,-4.28,2.29,1.80,0.05,2.04,3.94,-5.20,-1.40,1.78,-4.95,1.86,1.57,0.21,-1.47,1.60,-0.21
cross01_0.45_ab0_0.1_32,-2.39,0.38,1.45,0.37,0.50,2.22,-1.71,-0.84,1.60,-1.57,1.26,-1.13,0.28,-0.79,-1.09,1.64
cross01_0.45_ab0_0.15_78,-1.12,2.06,-0.31,-0.64,1.91,-0.16,-1.29,-0.41,-0.26,-1.34,0.88,0.66,-0.52,-0.57,0.69,0.42
